In [1]:
import re
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Define special tokens
SOS_TOKEN = "<sos>"
EOS_TOKEN = "<eos>"
PAD_TOKEN = "<pad>"
UNK_TOKEN = "<unk>"


conversation_pairs = [
    ("hello", "Hi there! Let's discuss Educational Technology."),
    ("what is educational technology?", "Educational Technology (EdTech) involves using digital tools to enhance learning."),
    ("how do adaptive learning platforms work?", "They personalize lessons based on a student’s progress using AI algorithms."),
    ("what is a flipped classroom?", "A flipped classroom lets students learn theory at home and do practical work in class."),
    ("which lms platforms are popular?", "Popular LMS platforms include Moodle, Canvas, and Blackboard."),
    ("how does virtual reality benefit education?", "VR immerses students in realistic simulations, improving engagement and retention."),
    ("why is gamification used in edtech?", "Gamification boosts motivation through points, badges, and leaderboards."),
    ("what are open educational resources?", "OER are free, openly licensed learning materials accessible to anyone."),
    ("how does mobile learning help?", "Mobile learning enables education anywhere using smartphones and tablets."),
    ("how does ai assist in education?", "AI provides personalized tutoring, grading automation, and adaptive learning."),
    ("why do teachers need training for edtech?", "Teacher training is crucial for effective tech integration in classrooms."),
    ("how is edtech success measured?", "Success is measured by student engagement, performance, and feedback."),
    ("what are the advantages of discussion forums?", "They allow online discussions, extending learning beyond the classroom."),
    ("how does edtech support students with disabilities?", "Assistive tech like screen readers and speech-to-text aids learning."),
    ("what is the role of cloud computing in edtech?", "Cloud storage provides remote access to educational content anytime."),
    ("what are common edtech challenges in rural areas?", "Limited internet, high costs, and lack of digital literacy are barriers."),
    ("what are micro-credentials?", "Micro-credentials certify mastery of specific skills, often through online courses."),
    ("why use online quizzes?", "Online quizzes offer instant feedback and help track student progress."),
    ("what are the latest edtech trends?", "AI-driven personalization, VR learning, and blockchain for credentials."),
    ("goodbye", "Goodbye! Keep exploring Educational Technology."),
]

# Build vocabulary function
def build_vocab(conversation_pairs, min_freq=1):
    word_freq = {}
    for (inp, out) in conversation_pairs:
        tokens_in = re.findall(r"\w+|\S", inp.lower())
        tokens_out = re.findall(r"\w+|\S", out.lower())

        for t in tokens_in + tokens_out:
            word_freq[t] = word_freq.get(t, 0) + 1

    vocab = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN]
    for w, freq in word_freq.items():
        if freq >= min_freq and w not in vocab:
            vocab.append(w)

    word2idx = {w: i for i, w in enumerate(vocab)}
    idx2word = {i: w for w, i in word2idx.items()}

    return vocab, word2idx, idx2word

# Initialize vocabulary
vocab, word2idx, idx2word = build_vocab(conversation_pairs)

# Sentence encoding function
def encode_sentence(sentence, word2idx, max_len=60):
    tokens = re.findall(r"\w+|\S", sentence.lower())
    encoded = [word2idx[SOS_TOKEN]]
    for t in tokens:
        encoded.append(word2idx.get(t, word2idx[UNK_TOKEN]))
    encoded.append(word2idx[EOS_TOKEN])

    return encoded + [word2idx[PAD_TOKEN]] * (max_len - len(encoded)) if len(encoded) < max_len else encoded[:max_len]

# Create dataset
def create_dataset(conversation_pairs, word2idx, max_len=60):
    data = [(encode_sentence(inp, word2idx, max_len), encode_sentence(out, word2idx, max_len)) for inp, out in conversation_pairs]
    return data

dataset = create_dataset(conversation_pairs, word2idx)

# Transformer-based chatbot model
class TransformerChat(nn.Module):
    def __init__(self, vocab_size, d_model=128, n_heads=4, num_layers=3, max_len=60):
        super().__init__()
        self.emb_inp = nn.Embedding(vocab_size, d_model)
        self.emb_out = nn.Embedding(vocab_size, d_model)

        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=n_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=256,
            dropout=0.1
        )

        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt):
        src_emb, tgt_emb = self.emb_inp(src).transpose(0, 1), self.emb_out(tgt).transpose(0, 1)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_emb.size(0)).to(src_emb.device)
        return self.fc_out(self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask)).transpose(0, 1)

# Dataset preparation
class ChatDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0], dtype=torch.long), torch.tensor(self.data[idx][1], dtype=torch.long)

# Training setup
batch_size = 4
chat_loader = DataLoader(ChatDataset(dataset), batch_size=batch_size, shuffle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerChat(len(vocab)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss(ignore_index=word2idx[PAD_TOKEN])
epochs = 150  # Increased epochs

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for src, tgt in chat_loader:
        src, tgt = src.to(device), tgt.to(device)
        logits = model(src, tgt[:, :-1])
        loss = criterion(logits.reshape(-1, len(vocab)), tgt[:, 1:].reshape(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(chat_loader):.4f}")

# Generate response function
def generate_reply(model, input_str, word2idx, idx2word, max_len=60):
    model.eval()
    src_tensor = torch.tensor(encode_sentence(input_str, word2idx, max_len), dtype=torch.long).unsqueeze(0).to(device)
    tgt_tokens = [word2idx[SOS_TOKEN]]

    for _ in range(max_len):
        tgt_tensor = torch.tensor(tgt_tokens, dtype=torch.long).unsqueeze(0).to(device)
        logits = model(src_tensor, tgt_tensor)
        next_token_id = logits[0, -1, :].argmax(dim=-1).item()
        tgt_tokens.append(next_token_id)
        if idx2word[next_token_id] == EOS_TOKEN:
            break

    return " ".join(idx2word[t] for t in tgt_tokens[1:] if idx2word[t] not in [PAD_TOKEN, EOS_TOKEN])

# Test chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    print("Bot:", generate_reply(model, user_input, word2idx, idx2word))


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/150 - Loss: 5.1244
Epoch 2/150 - Loss: 4.7738
Epoch 3/150 - Loss: 4.6458
Epoch 4/150 - Loss: 4.5418
Epoch 5/150 - Loss: 4.3921
Epoch 6/150 - Loss: 4.1644
Epoch 7/150 - Loss: 3.9271
Epoch 8/150 - Loss: 3.7265
Epoch 9/150 - Loss: 3.4906
Epoch 10/150 - Loss: 3.2718
Epoch 11/150 - Loss: 3.0767
Epoch 12/150 - Loss: 2.8436
Epoch 13/150 - Loss: 2.6679
Epoch 14/150 - Loss: 2.4869
Epoch 15/150 - Loss: 2.3087
Epoch 16/150 - Loss: 2.1736
Epoch 17/150 - Loss: 1.9987
Epoch 18/150 - Loss: 1.8486
Epoch 19/150 - Loss: 1.7385
Epoch 20/150 - Loss: 1.6138
Epoch 21/150 - Loss: 1.5365
Epoch 22/150 - Loss: 1.3929
Epoch 23/150 - Loss: 1.3415
Epoch 24/150 - Loss: 1.2165
Epoch 25/150 - Loss: 1.1510
Epoch 26/150 - Loss: 1.0565
Epoch 27/150 - Loss: 1.0023
Epoch 28/150 - Loss: 0.9275
Epoch 29/150 - Loss: 0.9085
Epoch 30/150 - Loss: 0.8331
Epoch 31/150 - Loss: 0.7732
Epoch 32/150 - Loss: 0.7517
Epoch 33/150 - Loss: 0.7038
Epoch 34/150 - Loss: 0.6591
Epoch 35/150 - Loss: 0.6251
Epoch 36/150 - Loss: 0.5929
E

KeyboardInterrupt: Interrupted by user